## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from functools import reduce
import random

## Utils

#### Test function

In [2]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [3]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


## Train data

In [4]:
suits = ['S1','S2','S3','S4','S5']
cards = ['C1','C2','C3','C4','C5']
distances = ['D1', 'D2', 'D3', 'D4', 'D5']
counted_suits = ['S1_count', 'S2_count', 'S3_count', 'S4_count']

In [5]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [6]:
train_dataset.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,4,9,2,1,2,2,4,7,2,8,0
1,1,4,3,6,1,12,3,11,2,7,0
2,1,11,4,1,3,7,4,11,2,1,2
3,2,9,2,4,3,6,1,9,4,9,3
4,1,8,2,4,2,11,2,2,2,1,0


In [7]:
data = train_dataset.drop('hand', axis=1)
train_labels = train_dataset['hand']

In [8]:
labels_vect = tf.keras.utils.to_categorical(train_labels)

#### Adding distance features

In [9]:
def get_distances(hand):
    distances_list = []
    for i, card in enumerate(hand):
        if (i < len(hand) - 1):
            distances_list.append(hand[i+1] - card)
        else:
            distances_list.append(13 - card + hand[0])
    return pd.Series(distances_list)

def with_distances(data):
    sorted = data[cards]
    sorted.values.sort()

    distances_df = sorted.apply(get_distances, axis=1)
    distances_df.columns = distances
    
    data_with_distances = pd.concat([data, distances_df], axis=1)
    return data_with_distances

In [10]:
data_with_distances = with_distances(data)
data_with_distances.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,D1,D2,D3,D4,D5
0,4,9,2,1,2,2,4,7,2,8,1,5,1,1,5
1,1,4,3,6,1,12,3,11,2,7,2,1,4,1,5
2,1,11,4,1,3,7,4,11,2,1,0,6,4,0,3
3,2,9,2,4,3,6,1,9,4,9,2,3,0,0,8
4,1,8,2,4,2,11,2,2,2,1,1,2,4,3,3


#### Adding similar suit counts

In [11]:
def get_suit_counts(hand):
    counts = hand.value_counts()
    res = np.zeros(4);
    for key in counts.keys():
        res[key - 1] = counts[key]
    return pd.Series(res)

def with_suit_counts(data):
    suit_counts = data[suits].apply(get_suit_counts, axis=1)
    suit_counts.columns = counted_suits
    return pd.concat([data, suit_counts], axis=1)

In [12]:
data_with_suit_counts = with_suit_counts(data_with_distances)
data_with_suit_counts.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,D1,D2,D3,D4,D5,S1_count,S2_count,S3_count,S4_count
0,4,9,2,1,2,2,4,7,2,8,1,5,1,1,5,0.0,3.0,0.0,2.0
1,1,4,3,6,1,12,3,11,2,7,2,1,4,1,5,2.0,1.0,2.0,0.0
2,1,11,4,1,3,7,4,11,2,1,0,6,4,0,3,1.0,1.0,1.0,2.0
3,2,9,2,4,3,6,1,9,4,9,2,3,0,0,8,1.0,2.0,1.0,1.0
4,1,8,2,4,2,11,2,2,2,1,1,2,4,3,3,1.0,4.0,0.0,0.0


#### One hot encoding

In [13]:
def one_hot_encoded(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

In [14]:
encoder = OneHotEncoder()
data_encoded = one_hot_encoded(data_with_suit_counts, suits, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(25010, 34)


,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,1,5,1,1,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,2,1,4,1,5,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,0,6,4,0,3,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,2,3,0,0,8,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1,2,4,3,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [15]:
def normalized(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data_normalized, data.drop(columns, axis=1)], axis=1)
    return to_return

In [16]:
mean_c = data_encoded[cards].mean().mean()
std_c = pd.Series(data_encoded[cards].values.flatten()).std()
mean_d = data_encoded[distances].mean().mean()
std_d = pd.Series(data_encoded[distances].values.flatten()).std()
mean_s = data_encoded[counted_suits].mean().mean()
std_s = pd.Series(data_encoded[counted_suits].values.flatten()).std()

data_normalized = normalized(data_encoded, cards, mean_c, std_c)
data_normalized = normalized(data_normalized, distances, mean_d, std_d)
data_normalized = normalized(data_normalized, counted_suits, mean_s, std_s)
data_normalized.head()

,S1_count,S2_count,S3_count,S4_count,D1,D2,D3,D4,D5,C1,...,10,11,12,13,14,15,16,17,18,19
0,-1.346861,1.885606,-1.346861,0.808117,-0.775959,1.163939,-0.775959,-0.775959,1.163939,0.537109,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.808117,-0.269372,0.808117,-1.346861,-0.290985,-0.775959,0.678965,-0.775959,1.163939,-0.796169,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-0.269372,-0.269372,-0.269372,0.808117,-1.260934,1.648914,0.678965,-1.260934,0.193990,1.070420,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,-0.269372,0.808117,-0.269372,-0.269372,-0.290985,0.193990,-1.260934,-1.260934,2.618863,0.537109,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.269372,2.963095,-1.346861,-1.346861,-0.775959,-0.290985,0.678965,0.193990,0.193990,0.270453,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
data_normalized.shape

(25010, 34)

## Neural Network

In [30]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(136, activation='relu', input_shape=(34,)))
model.add(keras.layers.Dropout(0.2))
#hidden
model.add(keras.layers.Dense(272, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(136, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(68, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(34, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=500, batch_size=512)

Epoch 1/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 34us/step - loss: 1.3352 - acc: 0.4649

Epoch 2/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 1.0250 - acc: 0.5171

Epoch 3/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.9785 - acc: 0.5560

Epoch 4/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.9300 - acc: 0.5948

Epoch 5/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.8614 - acc: 0.6334: 0s - loss: 0.8927 - 

Epoch 6/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 31us/step - loss: 0.8028 - acc: 0.6612

Epoch 7/500
25010/25010 [==============================]25010/25010 [==============================] -

25010/25010 [==============================]25010/25010 [==============================] - 1s 27us/step - loss: 0.0101 - acc: 0.9973

Epoch 57/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 27us/step - loss: 0.0116 - acc: 0.9965

Epoch 58/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.0095 - acc: 0.9974

Epoch 59/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.0080 - acc: 0.9978

Epoch 60/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.0072 - acc: 0.9976

Epoch 61/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.0083 - acc: 0.9976

Epoch 62/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 27us/step - loss: 0.0079

25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.0026 - acc: 0.9993

Epoch 112/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.0037 - acc: 0.9989

Epoch 113/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0029 - acc: 0.9992

Epoch 114/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.0026 - acc: 0.9992

Epoch 115/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 27us/step - loss: 0.0022 - acc: 0.9993

Epoch 116/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0025 - acc: 0.9994

Epoch 117/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 

25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 9.8134e-04 - acc: 0.9997

Epoch 166/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0027 - acc: 0.9995

Epoch 167/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.0016 - acc: 0.9995

Epoch 168/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 9.9850e-04 - acc: 0.9996

Epoch 169/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0013 - acc: 0.9997

Epoch 170/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 5.2549e-04 - acc: 0.9999

Epoch 171/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/s

25010/25010 [==============================]25010/25010 [==============================] - 1s 32us/step - loss: 0.0017 - acc: 0.9996

Epoch 220/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 0.0019 - acc: 0.9997

Epoch 221/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 32us/step - loss: 9.8838e-04 - acc: 0.9997

Epoch 222/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 0.0010 - acc: 0.9997

Epoch 223/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 8.4716e-04 - acc: 0.9998

Epoch 224/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 6.8784e-04 - acc: 0.9999

Epoch 225/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/s

25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 9.7204e-04 - acc: 0.9998

Epoch 274/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 6.8638e-04 - acc: 0.9998

Epoch 275/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 6.2762e-04 - acc: 0.9999

Epoch 276/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 4.6598e-04 - acc: 0.9998

Epoch 277/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 6.0050e-04 - acc: 0.9998

Epoch 278/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0012 - acc: 0.9998

Epoch 279/500
25010/25010 [==============================]25010/25010 [==============================] - 1

25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 5.4864e-04 - acc: 0.9998

Epoch 328/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 31us/step - loss: 7.0472e-04 - acc: 0.9998

Epoch 329/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 32us/step - loss: 0.0014 - acc: 0.9996

Epoch 330/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 31us/step - loss: 7.6429e-04 - acc: 0.9998

Epoch 331/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 33us/step - loss: 0.0017 - acc: 0.9996

Epoch 332/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 33us/step - loss: 3.0011e-04 - acc: 0.9999

Epoch 333/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30

25010/25010 [==============================]25010/25010 [==============================] - 1s 32us/step - loss: 2.6148e-04 - acc: 1.0000

Epoch 381/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 6.3374e-05 - acc: 1.0000

Epoch 382/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0013 - acc: 0.9997

Epoch 383/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 32us/step - loss: 0.0014 - acc: 0.9997

Epoch 384/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 6.4076e-04 - acc: 0.9998

Epoch 385/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0021 - acc: 0.9995

Epoch 386/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/s

25010/25010 [==============================]25010/25010 [==============================] - 1s 32us/step - loss: 0.0010 - acc: 0.9998

Epoch 435/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 35us/step - loss: 0.0011 - acc: 0.9998

Epoch 436/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 33us/step - loss: 3.8018e-04 - acc: 0.9999

Epoch 437/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 2.7668e-04 - acc: 0.9999

Epoch 438/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 27us/step - loss: 9.8847e-04 - acc: 0.9998

Epoch 439/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 7.2379e-04 - acc: 0.9998

Epoch 440/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28

25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 1.9990e-04 - acc: 1.0000

Epoch 488/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 4.6115e-04 - acc: 0.9999

Epoch 489/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 33us/step - loss: 3.4088e-04 - acc: 1.0000

Epoch 490/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 7.0514e-04 - acc: 0.9998

Epoch 491/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 9.9726e-04 - acc: 0.9996

Epoch 492/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 32us/step - loss: 4.9167e-04 - acc: 0.9999

Epoch 493/500
25010/25010 [==============================]25010/25010 [==============================]

In [28]:
#model.save('./model-35')

## Test data

In [18]:
test_dataset = pd.read_csv('test.csv', delimiter=',')
test_data = test_dataset.drop('id', axis=1)

In [19]:
test_labels = test_data.apply(calc_hand_label, axis=1)

In [20]:
test_data_with_distances = with_distances(test_data)
print('test_data_with_distances')
test_data_with_suit_counts = with_suit_counts(test_data_with_distances)
print('test_data_with_suit_counts')
test_data_encoded = one_hot_encoded(test_data_with_suit_counts, suits, encoder)
print('test_data_encoded')
test_data_normalized = normalized(test_data_encoded, cards, mean_c, std_c)
test_data_normalized = normalized(test_data_normalized, distances, mean_d, std_d)
test_data_normalized = normalized(test_data_normalized, counted_suits, mean_s, std_s)
print('test_data_normalized')

test_data_with_distances
test_data_with_suit_counts
test_data_encoded
test_data_normalized


In [21]:
test_data_normalized.to_csv('./test_data_with_distances_suit_counts.csv')

In [22]:
test_data_normalized.shape

(1000000, 34)

## Predictions

In [31]:
test_data_sample_predicted_10 = model.predict(np.array(test_data_normalized))
test_data_sample_predicted = np.array(list(map(np.argmax, test_data_sample_predicted_10)))

In [32]:
predictions_data = pd.concat([
    test_dataset,
    pd.DataFrame({'predictions': test_data_sample_predicted}),
    pd.DataFrame({'labels': test_labels}),
], axis=1)

In [33]:
wrong_answers = predictions_data[predictions_data['predictions'] != predictions_data['labels']]
wrong_answers.shape

(257, 13)

In [34]:
wrong_answers['labels'].value_counts()

4    201
0     51
5      5
Name: labels, dtype: int64

In [35]:
wrong_answers

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,predictions,labels
4202,4203,4,10,2,11,1,12,3,13,4,1,0,4
6823,6824,1,10,3,1,3,11,1,13,4,12,0,4
10252,10253,4,10,2,12,3,1,1,13,2,11,0,4
15059,15060,2,13,1,12,1,1,1,10,4,11,0,4
18740,18741,1,11,1,1,3,13,3,10,2,12,0,4
20512,20513,2,11,4,12,1,13,3,1,1,10,0,4
24133,24134,1,10,4,1,2,13,4,8,4,9,4,0
29282,29283,4,12,1,1,4,13,4,11,3,10,0,4
36911,36912,2,13,3,11,3,1,2,10,1,12,0,4
37362,37363,4,12,4,1,2,9,1,13,4,10,4,0


In [36]:
wrong_answers.drop(['id', 'predictions', 'labels'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

4202      10♠, J♦, Q♥, K♣, A♠
6823      10♥, A♣, J♣, K♥, Q♠
10252     10♠, Q♦, A♣, K♥, J♦
15059     K♦, Q♥, A♥, 10♥, J♠
18740     J♥, A♥, K♣, 10♣, Q♦
20512     J♦, Q♠, K♥, A♣, 10♥
24133     10♥, A♠, K♦, 8♠, 9♠
29282     Q♠, A♥, K♠, J♠, 10♣
36911     K♦, J♣, A♣, 10♦, Q♥
37362     Q♠, A♠, 9♦, K♥, 10♠
50026     J♠, K♥, A♣, 10♦, Q♣
50832     Q♦, 10♥, A♦, K♥, J♠
51825     Q♥, 10♥, A♠, K♣, J♣
52123     Q♣, A♦, 9♣, 10♣, K♦
57315     10♣, K♣, Q♣, 2♥, J♦
57633     J♠, Q♦, K♠, 10♠, A♠
59297     A♥, 10♣, Q♥, J♥, 9♥
59999     Q♦, A♥, J♠, 10♥, K♣
62284     A♣, 10♥, J♣, Q♠, K♥
65236     10♥, Q♣, A♦, K♦, J♠
69065     Q♣, J♥, A♦, 10♦, K♦
70018     A♦, 10♠, Q♦, J♦, K♦
72302     Q♠, K♠, 9♦, 10♥, A♣
78123     10♦, A♥, J♥, K♣, Q♥
84987     J♦, 10♠, K♥, A♣, Q♥
85179     Q♦, K♣, A♣, J♦, 10♥
95994     10♣, K♥, Q♥, J♠, A♦
98876     10♠, K♠, 9♦, Q♠, A♠
103163    K♥, A♣, Q♦, J♣, 10♥
105032    10♠, K♠, A♣, J♦, Q♦
                 ...         
884950    Q♦, J♦, 10♦, K♦, 2♦
886751    K♥, 10♦, A♠, Q♠, J♣
886871    